# Test the new structure of grid search

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load libraries
import gs_model
import json

## 1 Load data according to the specified region

In [3]:
# load region data
%%time
region='sweden-west'
sw_n = gs_model.RegionDataPrep(region=region)
sw_n.load_zones_odm()
sw_n.load_geotweets()
# blind-guess
sw_n.kl_baseline_compute()
print('Baseline kl measure', sw_n.kl_baseline)


odm (8791225,) 3339597.621275899
Reindexing...
odm (8785296,)
Calculating distances...
Calculating distances between zones...
Calculating distance quantiles...
Calculating quantiles...
Wall time: 44.5 s


## 2 Generate visits

In [6]:
%%time
visits = gs_model.VisitsGeneration(region=region, bbox=sw_n.bbox,
                                   zones=sw_n.zones, odm=sw_n.gt_odm,
                                   distances=sw_n.distances, distance_quantiles=sw_n.distance_quantiles,
                                   gt_dms=sw_n.dms)
p = 0.3
beta = 0.03
gamma = 0.8
divergence_measure = visits.visits_gen(geotweets=sw_n.tweets_calibration, home_locations=sw_n.home_locations,
                                       p=p, gamma=gamma, beta=beta)
print(divergence_measure)

In [15]:
# append the result to the gridsearch file
with open(ROOT_dir + '/results/gridsearch.txt', 'a') as outfile:
    json.dump({'region': region, 'p': p, 'beta': beta, 'gamma': gamma,
               'kl-baseline': sw_n.kl_baseline, 'kl': divergence_measure}, outfile)
    outfile.write('\n')

In [2]:
os.path.realpath(__file__)

NameError: name '__file__' is not defined